## 千兆病理图像癌细胞转移检测
## Detecting Cancer Metastases on Gigapixel Pathology Images

In [1]:
import openslide
import numpy as np
from pylab import *

# 读取包含有肿瘤区域的大图（全切片病理图像）
origin_images_path = "/atlas/home/zwpeng/paper_rebuild/camelyon/train/tumor/origin_images/Tumor_005.tif"
origin_slide = openslide.open_slide(origin_images_path)

# 读取该肿瘤区域的标注图
annotation_images_path = "/atlas/home/zwpeng/paper_rebuild/camelyon/train/tumor/annotation_images/Tumor_005_Mask.tif"
mask_slide = openslide.open_slide(annotation_images_path)

In [2]:
# origin_slide.level_count,origin_slide.level_dimensions,origin_slide.level_downsamples     #查看病理图片的金字塔结构

In [3]:
# mask_slide.level_count,mask_slide.level_dimensions,mask_slide.level_downsamples     #查看标注图片的金字塔结构

### 首先找到感兴趣区域

In [2]:
# 方法三：通过获取每一块区域的像素值R G B各自的平均数，然后相减，设置一个阈值，将噪点（墨迹）和有效区　分开

from pylab import *
import numpy as np

# 感兴趣区域锁定函数
def locate_ROI(origin_slide,level=6):

    origin_widths,origin_heights = origin_slide.dimensions

    object_widths,object_heights = origin_slide.level_dimensions[level]

    rgb_list_y = list()
    rgb_list_x = list()
    rgb_var_x = []
    rgb_var_y = []
    rgb_var_xi = []
    rgb_var_yi = []

    # 寻找有效区域的y值、高度
    for k in range(100):
        slide = origin_slide.read_region((0, k*origin_heights//100), level, (object_widths, object_heights//50)) 
        slide_arr = array(slide.convert("RGB"))
        arrR = np.mean(slide_arr[:,:,:1])
        arrG = np.mean(slide_arr[:,:,1:2])
        arrB = np.mean(slide_arr[:,:,2:3])
        rgb_list_y.append((arrR,arrG,arrB))
    for i,rgbVar in enumerate(rgb_list_y):
        rgb_var_y.append(np.var(rgbVar))
        if np.var(rgbVar)>=1:
            rgb_var_yi.append(i)

#     print(rgb_var_yi)
    effective_y = min(rgb_var_yi)*origin_heights//100        #有效区域的左上顶点y坐标找到了
    effective_heights = (max(rgb_var_yi)-min(rgb_var_yi))*origin_heights//100 + origin_heights//50  #有效区域的高度也出来了
#     print("有效区域的ｙ值是：%d" %effective_y, "有效区域的高度是：%d" %effective_heights)

    # 寻找有效区域的x值、宽度
    for j in range(100):
        slide = origin_slide.read_region((j*origin_widths//100, effective_y), level, 
                                          (object_widths//50, effective_heights//62))     # 循环顺序读取50宽的区域
    #     slide = origin_slide.read_region((j*origin_widths//100, 0), level, 
    #                                       (object_widths//50, object_heights))     # 循环顺序读取50宽的区域

        slide_arr = array(slide.convert("RGB"))
        arrR = np.mean(slide_arr[:,:,:1])
        arrG = np.mean(slide_arr[:,:,1:2])
        arrB = np.mean(slide_arr[:,:,2:3])
        rgb_list_x.append((arrR,arrG,arrB))
    for i,rgbVar in enumerate(rgb_list_x):
        rgb_var_x.append(np.var(rgbVar))
        if np.var(rgbVar)>=2:
            rgb_var_xi.append(i)

#     print(rgb_var_xi)
    effective_x = min(rgb_var_xi)*origin_widths//100        # 有效区域的左上顶点y坐标找到了
    effective_widths = (max(rgb_var_xi) - min(rgb_var_xi))*origin_widths//100 + origin_widths//50  # 有效区域的宽度也出来了
#     print("有效区域的ｘ值是：%d" %effective_x, "有效区域的宽度是：%d" %effective_widths)
#     plt.plot(range(100), rgb_var_y[:100], label='rgb_var_curve')
    # plt.plot(range(100), rgb_var_x[:100], label='rgb_var_curve')
#     plt.legend()
#     plt.show()
    return effective_x,effective_y,effective_widths,effective_heights

In [3]:
effective_x,effective_y,effective_widths,effective_heights = locate_ROI(origin_slide)

In [4]:
effective_x,effective_y,effective_widths,effective_heights

(18580, 125199, 41071, 50518)

这个有效区域，经过手动调试，起点坐标：(17600,124700) ，区域长宽：(44800,57600)



### mask 的有效区域，定位比较容易

In [5]:
from pylab import *
import numpy as np

def locate_ROI_mask(mask_slide,mask_level=7):

    # level0　的尺寸
    mask_widths, mask_heights = mask_slide.dimensions
    # level7 的尺寸
    mask_level_widths, mask_level_heights = mask_slide.level_dimensions[mask_level]

    mask_level_slide = mask_slide.read_region((0, 0), mask_level, (mask_level_widths, mask_level_heights))
    mask_level_slide_gray = mask_level_slide.convert("L")
    mask_level_slide_arr = array(mask_level_slide_gray)

    mask_y, mask_x = nonzero(mask_level_slide_arr)  # 因为mask是黑白图，只需直接获得非零像素的坐标
    # mask_x, mask_y
    tumor_leftup_x = (min(mask_x)-1) * int(mask_slide.level_downsamples[mask_level])
    tumor_leftup_y = (min(mask_y)-1) * int(mask_slide.level_downsamples[mask_level])
    tumor_rightdown_x = (max(mask_x)+1) * int(mask_slide.level_downsamples[mask_level])
    tumor_rightdown_y = (max(mask_y)+1) * int(mask_slide.level_downsamples[mask_level])
    
#     print(tumor_leftup_x,tumor_leftup_y,tumor_rightdown_x,tumor_rightdown_y)
    mask_effective_widths = tumor_rightdown_x - tumor_leftup_x
    mask_effective_heights = tumor_rightdown_y - tumor_leftup_y
    
#     mask_tumor_area = ((max(mask_x)-min(mask_x)+2)*int(mask_slide.level_downsamples[mask_level]), 
#                        (max(mask_y)-min(mask_y)+2)*int(mask_slide.level_downsamples[mask_level]))
#     print(mask_tumor_area)        # mask区域的长宽
    return tumor_leftup_x,tumor_leftup_y,mask_effective_widths,mask_effective_heights

In [6]:
tumor_leftup_x,tumor_leftup_y,mask_effective_widths,mask_effective_heights = locate_ROI_mask(mask_slide)

In [7]:
tumor_leftup_x,tumor_leftup_y,mask_effective_widths,mask_effective_heights

(57600, 136320, 2048, 3712)

In [10]:
# mask_slide.read_region((tumor_leftup_x,tumor_leftup_y),0,(mask_effective_widths,mask_effective_heights)) 

In [8]:
import numpy as np
from PIL.Image import Image
from pylab import *
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

# 有效区域（感兴趣区域），所有细胞聚集区
# effective_area = (effective_x, effective_y)
# effective_area_size = (effective_widths, effective_heights)
# 有效区域，所有标注的细胞聚集区
# mask_slide.read_region(mask_tumor_start, 0, mask_tumor_area) 
# random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))  # 这一区域就是随机读到的区域，
                                                            #接下来就要判断这个区域在哪里，应该给他贴上什么标签

# 随机生成一个０到１之间的数，判断是否大于0.5,如果大于0.5,就从tumor区（mask）获取随机点（产生随机图片）；
# 如果小于0.5,就从normal区获取随机点，产生的随机图片需要判断是否取到了tumor
# for i in range(30):    # 这个其实就是 batch_size
#     widths, heights = 299, 299
def data_generator(widths=299,heights=299):
    while True:
        random_num = np.random.random(1)
        print("0到１之间的随机数是：%s"%random_num)

        if random_num > 0.5:
            # 定义随机坐标,一定要取到一张含有tumor的图片
            random_x = np.random.randint(tumor_leftup_x, tumor_leftup_x + mask_effective_widths - widths)  
            random_y = np.random.randint(tumor_leftup_y, tumor_leftup_y + mask_effective_heights - heights)
    #             print("取tumor随机点坐标是：%d,%d"%(random_x,random_y))
            random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
            random_img_mask_arr = array(random_img_mask.convert("L"))
            random__img_y, random_img_x = nonzero(random_img_mask_arr)
            while len(random_img_x)==0:
                random_x = np.random.randint(tumor_leftup_x, tumor_leftup_x + mask_effective_widths - widths)
                random_y = np.random.randint(tumor_leftup_y, tumor_leftup_y + mask_effective_heights - heights)
    #                 print("取tumor随机点坐标是：%d,%d"%(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)

            #*********************上面这个 while 循环结束后，就产生了一个合格的坐标点*********************#
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))
#             f,axes = plt.subplots(1,2)
#             plt.subplot(1,2,1)
#             plt.imshow(origin_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.set_title('tumor_image')
#             plt.subplot(1,2,2)
#             plt.imshow(mask_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.set_title('tumor_mask')
#             plt.show()

            #***接下来就给他贴标签，并处理成训练所需的数据结构***#
            random_img_arr = array(random_img.convert("RGB"))
            x = np.expand_dims(random_img_arr, axis=0)/255.
            y = to_categorical(0,2)    
        else:
            # 定义随机坐标，一定要取到一张不含有tumor的normal图片
            random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)   # 大图上,nomal有效区的起点和终点
            random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
    #             print("取normal随机点坐标是：%d,%d"%(random_x,random_y))
            random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
            random_img_mask_arr = array(random_img_mask.convert("L"))
            random__img_y, random_img_x = nonzero(random_img_mask_arr)
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))

#             print("随机情况",len(random_img_x),(array(random_img.convert("RGB"))).std())

            while (array(random_img.convert("RGB"))).std()<20.0:
                random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)
                random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
    #                 print("取normal随机点坐标是：%d,%d" %(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)
                random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))

#             print("颜色检测情况",len(random_img_x),(array(random_img.convert("RGB"))).std())

            while len(random_img_x) != 0:
                random_x = np.random.randint(effective_x,effective_x+effective_widths-widths)
                random_y = np.random.randint(effective_y,effective_y+effective_heights-heights)
    #                 print("取normal随机点坐标是：%d,%d" %(random_x,random_y))
                random_img_mask = mask_slide.read_region((random_x,random_y),0,(widths,heights))
                random_img_mask_arr = array(random_img_mask.convert("L"))
                random__img_y, random_img_x = nonzero(random_img_mask_arr)
                random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))

#             print("非tumor区检测情况",len(random_img_x), (array(random_img.convert("RGB"))).std())

            #*********************上面这个 while 循环结束后，就产生了一个合格的坐标点*********************#
            random_img = origin_slide.read_region((random_x,random_y),0,(widths,heights))
#             f,axes = plt.subplots(1,2)
#             plt.subplot(1,2,1)
#             plt.imshow(origin_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.title('normal_image')
#             plt.subplot(1,2,2)
#             plt.imshow(mask_slide.read_region((random_x,random_y),0,(widths,heights)))
#             plt.title('normal_mask')
#             plt.show()
    #         f,axes = plt.subplots(1,2,figsize=(20,20))
    #         ax = axes.ravel()
    #         ax[0].imshow(origin_slide.read_region((random_x,random_y),0,(widths,heights)))
    #         ax[0].set_title('normal_image')
    #         ax[1].imshow(mask_slide.read_region((random_x,random_y),0,(widths,heights)))
    #         ax[1].set_title('normal_mask')
    #         f
            #***接下来就给他贴标签，并处理成训练所需的数据结构***#
            random_img_arr = array(random_img.convert("RGB"))
            x = np.expand_dims(random_img_arr, axis=0)/255.
            y = to_categorical(1,2) 
        yield (x,y)

Using TensorFlow backend.


In [23]:
a = data_generator()

In [33]:
a.__next__() 　　　　# yield要用next调用

0到１之间的随机数是：[ 0.79746816]


(array([[[[ 0.7254902 ,  0.42745098,  0.58039216],
          [ 0.74509804,  0.44705882,  0.59607843],
          [ 0.6745098 ,  0.39215686,  0.5372549 ],
          ..., 
          [ 0.25098039,  0.10980392,  0.25882353],
          [ 0.23921569,  0.09803922,  0.24705882],
          [ 0.25098039,  0.10980392,  0.25098039]],
 
         [[ 0.77254902,  0.4745098 ,  0.62352941],
          [ 0.7254902 ,  0.43529412,  0.58039216],
          [ 0.61176471,  0.3372549 ,  0.48235294],
          ..., 
          [ 0.21568627,  0.07843137,  0.24313725],
          [ 0.19607843,  0.07058824,  0.23137255],
          [ 0.20784314,  0.08235294,  0.23529412]],
 
         [[ 0.80784314,  0.50980392,  0.65882353],
          [ 0.70588235,  0.41568627,  0.56078431],
          [ 0.54901961,  0.28235294,  0.42745098],
          ..., 
          [ 0.18823529,  0.06666667,  0.23137255],
          [ 0.17647059,  0.05490196,  0.21568627],
          [ 0.19215686,  0.0745098 ,  0.22352941]],
 
         ..., 
         [

In [12]:
%time example_X, example_y  = next(data_generator())    # yield要用next调用

0到１之间的随机数是：[ 0.54043864]
CPU times: user 32.1 ms, sys: 15.9 ms, total: 48 ms
Wall time: 48.6 ms


In [13]:
example_X.shape,example_y.shape

((1, 299, 299, 3), (1, 2))

### 注意这里的第一个数表示batch_size


接下来可以开始训练了。

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD


def Creat_InvepV3(num_lay_out=64,classes=2,dropout=0.5):
    img_width, img_height = 299, 299
    base_model = InceptionV3(weights="imagenet", include_top=False,
                             input_shape=(img_width, img_height, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(num_lay_out)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(classes)(x)
    x = BatchNormalization()(x)
    x = Activation('softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=SGD(lr=0.002, momentum=0.9, decay=0.5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [9]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import SGD


def Creat_InvepV3(num_lay_out=1024,classes=2,dropout=0.5):
    img_width, img_height = 299, 299
    base_model = InceptionV3(weights="imagenet", include_top=False,
                             input_shape=(img_width, img_height, 3))
    x = base_model.output
    # x = Flatten()(x)
    # x = Dense(num_lay_out)(x)
    # x = BatchNormalization()(x)
    # x = Activation('relu')(x)
    # x = Dropout(dropout)(x)
    # x = Dense(classes)(x)
    # x = BatchNormalization()(x)
    # x = Activation('softmax')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_lay_out, activation='relu')(x)
    predictions = Dense(classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=SGD(lr=0.05, momentum=0.9, decay=0.5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
from __future__ import division
import keras
from keras.callbacks import CSVLogger
import gc

def train_model():
    keras.backend.tensorflow_backend.clear_session()
    epochs = 100
    model = Creat_InvepV3()
    csvlogger = CSVLogger('training0817.log', append=True)
    model.fit_generator(data_generator(),
                        steps_per_epoch=100,
                        epochs=epochs,
                        validation_data=data_generator(),
                        validation_steps=20,
                        verbose=0,
                        workers=2,
                        max_q_size=1,
                        callbacks=[csvlogger])
    model.save('model0817.h5')
    gc.collect()
train_model()

0到１之间的随机数是：[ 0.97810762]


Exception in thread Thread-23:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/keras/engine/training.py", line 606, in data_generator_task
    generator_output = next(self._generator)
ValueError: generator already executing



ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None